# Using CDP Databases
Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

### Connecting to the database

CDP Seattle uses Firebase's 'Cloud Firestore' to store our data. However, a properly setup database host and associated database module _should_ have the same functionality.

Here is how to connect to the Seattle database for **read only** operations.

**Note:** This notebook connects to the staging instance of Seattle's Firestore database. To use production data, connect to `cdp-seattle`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase

db = CloudFirestoreDatabase("stg-cdp-seattle")
db

<CloudFirestoreDatabase [stg-cdp-seattle]>

### Retrieving a single item
If you know the id of an item in a table, please use the `select_row_by_id` function provided.

In [2]:
event = db.select_row_by_id(table="event", id="afc636e3-1ce5-4f5a-88b0-58b49a70124e")
event

{'event_id': 'afc636e3-1ce5-4f5a-88b0-58b49a70124e',
 'source_uri': 'http://www.seattlechannel.org/FullCouncil?videoid=x104832',
 'legistar_event_id': 3983,
 'event_datetime': datetime.datetime(2019, 6, 10, 14, 0),
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/6/3983_A_City_Council_19-06-10_Full_Council_Meeting_Agenda.pdf',
 'minutes_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/6/3983_M_City_Council_19-06-10_Full_Council_Meeting_Minutes.pdf',
 'video_uri': 'http://video.seattle.gov:8080/media/council/council_061019_2021945V.mp4',
 'created': datetime.datetime(2019, 7, 8, 1, 53, 56, 538108),
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=3983&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'body_id': '1b73c822-39e7-4252-aedb-4d22de5e4efb'}

### Retrieving many items from a table

You may not know the id's of items you are looking for. In that case, use the `select_rows_as_list` function provided.

In [3]:
events = db.select_rows_as_list(table="event")
events[0]

{'event_id': '15ce0a20-3688-4ebd-bf3f-24f6e8d12ad9',
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/6/3981_A_Human_Services%2C_Equitable_Development%2C_and_Renter_Rights_Committee_19-06-11_Committee_Agenda.pdf',
 'minutes_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/6/3981_M_Human_Services%2C_Equitable_Development%2C_and_Renter_Rights_Committee_19-06-11_Committee_Minutes.pdf',
 'video_uri': 'http://video.seattle.gov:8080/media/council/hum_061119_2591921V.mp4',
 'created': datetime.datetime(2019, 7, 8, 2, 4, 33, 697012),
 'body_id': 'f8579c50-ddce-44f3-8304-7eec6085abcd',
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=3981&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'source_uri': 'http://www.seattlechannel.org/mayor-and-council/city-council/2018/2019-human-services-equitable-development-and-renter-rights-committee?videoid=x104867',
 'legistar_event_id': 3981,
 'event_datetime': datetime.datetime(2019, 6, 

### Joining with other tables

In the above event results, notice that a `body_id` is returned for each event. To attach body details to this we can use the python package `pandas` and query the `body` table. Let's first put each of the query results into `pandas.DataFrame` objects.

In [4]:
import pandas as pd

In [5]:
events = pd.DataFrame(events)
events.head()

,agenda_file_uri,body_id,created,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
0,http://legistar2.granicus.com/seattle/meetings...,f8579c50-ddce-44f3-8304-7eec6085abcd,2019-07-08 02:04:33.697012,2019-06-11 14:00:00,15ce0a20-3688-4ebd-bf3f-24f6e8d12ad9,3981,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...
1,http://legistar2.granicus.com/seattle/meetings...,c17eacbf-b40c-4160-b2b8-5a3e58e9124f,2019-07-08 01:50:00.679099,2019-06-11 09:30:00,3571c871-6f7d-41b5-85d1-ced0589f9220,3982,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...
2,http://legistar2.granicus.com/seattle/meetings...,aa421a20-a4c5-4893-94e4-ce8cbb60b05d,2019-07-08 01:23:09.749805,2019-06-24 09:30:00,44af0ef0-cdfc-4cf1-924f-913065a0a21e,3987,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pa...
3,http://legistar2.granicus.com/seattle/meetings...,f8579c50-ddce-44f3-8304-7eec6085abcd,2019-07-08 02:23:53.312312,2019-06-25 14:00:00,4661ee79-b85f-44b2-b784-9aa97372b35c,4012,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...
4,http://legistar2.granicus.com/seattle/meetings...,1b73c822-39e7-4252-aedb-4d22de5e4efb,2019-07-08 00:52:09.898324,2019-06-17 14:00:00,56af907c-51cb-4364-bbbf-e686e1244f27,4002,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...


In [6]:
bodies = db.select_rows_as_list("body")
bodies = pd.DataFrame(bodies)
bodies.head()

,body_id,created,description,name
0,0d55e511-d74d-42ef-96ef-bd215741747b,2019-07-07 23:35:17.416092,None,"Housing, Health, Energy, and Workers’ Rights C..."
1,1b73c822-39e7-4252-aedb-4d22de5e4efb,2019-07-07 23:46:03.311538,None,City Council
2,21ed798f-ef5a-4b25-b936-ce12130e97e8,2019-07-08 01:36:05.787398,None,"Civic Development, Public Assets, and Native C..."
3,47b72e68-1da1-4160-8950-51f1eff77429,2019-07-07 23:33:42.647687,None,Sustainability and Transportation Committee
4,7f5f8ea0-b387-449d-8f5a-515e90f180d7,2019-07-08 01:52:46.040526,None,"Governance, Equity, and Technology Committee"


In [7]:
expanded_event_details = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
expanded_event_details.head()

,agenda_file_uri,body_id,created_event,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description,name
0,http://legistar2.granicus.com/seattle/meetings...,f8579c50-ddce-44f3-8304-7eec6085abcd,2019-07-08 02:04:33.697012,2019-06-11 14:00:00,15ce0a20-3688-4ebd-bf3f-24f6e8d12ad9,3981,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...,2019-07-08 02:04:33.183759,None,"Human Services, Equitable Development, and Ren..."
1,http://legistar2.granicus.com/seattle/meetings...,f8579c50-ddce-44f3-8304-7eec6085abcd,2019-07-08 02:23:53.312312,2019-06-25 14:00:00,4661ee79-b85f-44b2-b784-9aa97372b35c,4012,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...,2019-07-08 02:04:33.183759,None,"Human Services, Equitable Development, and Ren..."
2,http://legistar2.granicus.com/seattle/meetings...,c17eacbf-b40c-4160-b2b8-5a3e58e9124f,2019-07-08 01:50:00.679099,2019-06-11 09:30:00,3571c871-6f7d-41b5-85d1-ced0589f9220,3982,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...,2019-07-08 01:41:43.471524,None,"Civil Rights, Utilities, Economic Development,..."
3,http://legistar2.granicus.com/seattle/meetings...,c17eacbf-b40c-4160-b2b8-5a3e58e9124f,2019-07-08 01:41:43.721244,2019-06-25 09:30:00,8681c031-09cf-4828-98f3-202417205a6f,4009,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...,2019-07-08 01:41:43.471524,None,"Civil Rights, Utilities, Economic Development,..."
4,http://legistar2.granicus.com/seattle/meetings...,aa421a20-a4c5-4893-94e4-ce8cbb60b05d,2019-07-08 01:23:09.749805,2019-06-24 09:30:00,44af0ef0-cdfc-4cf1-924f-913065a0a21e,3987,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pa...,2019-07-07 22:56:33.472965,None,Council Briefing


`left_on` refers to the column name in the dataframe calling the operation.
In this case, the column to merge on is `body_id` in the events results.

Similarly, `right_on` refers to the column name in the dataframe to be passed to the operation.
In this case, the column to merge on is `id` in the bodies results.

`suffixes` is a tuple to use for adding suffixes to any columns with the same name between the two dataframes.
Commonly for CDP query results, these are columns such as `created`, which provide a `datetime` value for when that row was stored in the database.

Please refer to `pandas.DataFrame.merge` documentation for more details.

[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

### Filtering

You may notice that the function: `select_rows_as_list` allows for additional parameters to be passed: `filters`, `order_by`, and `limit`. Unfortunately, at this time, `filters` is not available for the open access portions of the API. So while you can provide them to the function, they are not actually used. Because of this, you must do filtering on your end. Fortunately however, `pandas` works well for these types of operations.

[stackoverflow](https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas)

In [8]:
fnc = "Finance and Neighborhoods Committee"
fnc_events = expanded_event_details.loc[expanded_event_details["name"] == fnc]
fnc_events

,agenda_file_uri,body_id,created_event,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description,name
17,http://legistar2.granicus.com/seattle/meetings...,914582fc-7c79-4c6e-80fe-647fd96c3709,2019-07-08 01:56:04.332503,2019-06-12 14:00:00,95145384-360b-46a5-9786-4a3d76ef25fb,3980,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-07-08 00:35:24.391795,None,Finance and Neighborhoods Committee
18,http://legistar2.granicus.com/seattle/meetings...,914582fc-7c79-4c6e-80fe-647fd96c3709,2019-07-08 00:35:24.712314,2019-06-26 14:00:00,d2536caf-1b3f-4830-a79c-e7ffe3f5975c,4010,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-07-08 00:35:24.391795,None,Finance and Neighborhoods Committee
